In [9]:
# Utils packages
import ast
from pathlib import Path, PurePath
from typing import Union

# Data collector packages
from hentai import Utils, Hentai, Option

# Analysis packages
import pandas as pd
from PIL import Image
from tqdm import tqdm

# Compute packages
import numpy as np
import tensorflow as tf

## Data collection

We will use the information and media collected on the nhentai.net website. Recording each entry in a 'metadata.csv' file that aggregates information about the hentai, including links to download pages.

In [10]:
N_SAMPLE = 11 # Number of sample to refetch from hentai.
DATA_PATH = "data" # Directory to drop collected data.

In [11]:
def random_hentai(_: int) -> pd.Series:
    return pd.Series(Utils.get_random_hentai().dictionary(Option.all()))

In [12]:
# Query the nhentai.net API to get N number of random hentai
hentais_df = pd.Series(range(N_SAMPLE), dtype="Int64")
hentais_df = hentais_df.apply(random_hentai)

# The metadata CSV file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "raw" / "metadata.csv"

# Create a CSV file with a header only if the file does not exist
if not metadata_path.is_file():
    hentais_df.to_csv(metadata_path, index=False, header="column_names")
else:
    hentais_df.to_csv(metadata_path, index=False, mode="a", header=False)

print("Number of collected hentais: ", len(hentais_df))

Number of collected hentais:  11


## Read dataset file

In [ ]:
hentais_df = pd.read_csv(
    Path(DATA_PATH) / "raw" / "metadata.csv",
    converters={
        column_name: ast.literal_eval
        for column_name in [
            "tag",
            "group",
            "parody",
            "character",
            "artist",
            "category",
            "image_urls"
        ]
    }
)
hentais_df

## Remove duplicates

In [6]:
hentais_df = hentais_df.drop_duplicates("id")
hentais_df.to_csv(metadata_path, index=False, header="column_names")

## Download images

In [16]:
def download_hentai(hentai: pd.Series) -> Hentai:
    # A hentai will be stored based on the gallery ID instead of the medai ID
    hentai_path = Path(DATA_PATH) / "raw" / str(hentai.id)

    # Since downloading a hentai is an expensive operation, we only
    # download when the hentai ID directory does not exist
    if not hentai_path.is_dir():
        hentai = Hentai(hentai.id)
        hentai.download(folder=hentai_path, progressbar=True)
        return True

    return False

downloaded_hentais_df = hentais_df.apply(download_hentai, axis=1)

data\raw\91511


TypeError: exceptions must derive from BaseException

In [20]:
print("Number of downloaded hentais :", downloaded_hentais_df.sum())

Number of downloaded hentais : 13


## Remove corrupted images

In [21]:
def hentai_filenames(hentai: pd.Series):
    return [
        str(PurePath(str(hentai["id"])) / Path(url).name)
        for url in hentai["image_urls"]
    ]

hentais_df = hentais_df.assign(filenames=hentais_df.apply(hentai_filenames, axis=1))
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages,filenames
0,352167,Boku wa Chiisana Succubus no Shimobe - I am a ...,NaN,https://nhentai.net/g/352167,https://nhentai.net/api/gallery/352167,1872914,1616460447,20119,"[wings, garter belt, group, uncensored, story ...",[],[],[],"['english', 'translated']",[tamano kedama],[manga],https://t.nhentai.net/galleries/1872914/cover.jpg,https://t.nhentai.net/galleries/1872914/thumb.jpg,[https://i.nhentai.net/galleries/1872914/1.jpg...,195,"[352167\1.jpg, 352167\2.jpg, 352167\3.jpg, 352..."
1,212548,Syoku,NaN,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],[occultic nine],[ririka nishizono],"['english', 'translated']",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33,"[212548\1.jpg, 212548\2.jpg, 212548\3.jpg, 212..."
2,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,NaN,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],[],[],['japanese'],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166,"[57170\1.jpg, 57170\2.jpg, 57170\3.jpg, 57170\..."
3,75495,SHIRONA HAZARD,NaN,https://nhentai.net/g/75495,https://nhentai.net/api/gallery/75495,490119,1403996013,191,[big breasts],[draw go],[pokemon],[cynthia],['japanese'],[souichi],[doujinshi],https://t.nhentai.net/galleries/490119/cover.jpg,https://t.nhentai.net/galleries/490119/thumb.jpg,"[https://i.nhentai.net/galleries/490119/1.jpg,...",23,"[75495\1.jpg, 75495\2.jpg, 75495\3.jpg, 75495\..."
4,229284,2205-nen no Afterglow,NaN,https://nhentai.net/g/229284,https://nhentai.net/api/gallery/229284,1206999,1522895460,101,"[eyepatch, sole male, sole female]",[frozen world],[touken ranbu],[shokudaikiri mitsutada],['japanese'],[kanaso],[doujinshi],https://t.nhentai.net/galleries/1206999/cover.jpg,https://t.nhentai.net/galleries/1206999/thumb.jpg,[https://i.nhentai.net/galleries/1206999/1.jpg...,47,"[229284\1.jpg, 229284\2.png, 229284\3.jpg, 229..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,8414,CRIMsON plus,NaN,https://nhentai.net/g/8414,https://nhentai.net/api/gallery/8414,23918,1403982375,27,[],[ura fmo],[tsukihime],[],['japanese'],[fumio],[doujinshi],https://t.nhentai.net/galleries/23918/cover.jpg,https://t.nhentai.net/galleries/23918/thumb.jpg,"[https://i.nhentai.net/galleries/23918/1.jpg, ...",22,"[8414\1.jpg, 8414\2.jpg, 8414\3.jpg, 8414\4.jp..."
996,301679,Danchi no Shoujo,NaN,https://nhentai.net/g/301679,https://nhentai.net/api/gallery/301679,1570827,1581719888,1058,"[dark skin, lolicon, prostitution, sole female...",[],[],[],"['chinese', 'translated']",[misao.],[manga],https://t.nhentai.net/galleries/1570827/cover.jpg,https://t.nhentai.net/galleries/1570827/thumb.jpg,[https://i.nhentai.net/galleries/1570827/1.jpg...,31,"[301679\1.jpg, 301679\2.jpg, 301679\3.jpg, 301..."
997,316439,Shiokaze ni Fuka Retanode,NaN,https://nhentai.net/g/316439,https://nhentai.net/api/gallery/316439,1656670,1591805811,2760,"[sole male, hairy, lolicon, niece, sole female...",[],[],[],"['chinese', 'translated']",[syamonabe],[manga],https://t.nhentai.net/galleries/1656670/cover.png,https://t.nhentai.net/galleries/1656670/thumb.png,[https://i.nhentai.net/galleries/1656670/1.png...,21,"[316439\1.png, 316439\2.png, 316439\3.png, 316..."
998,329359,Ore no Kirai na Senpai | 我的讨人厌前辈 Ch. 1-2,NaN,https://nhentai.net/g/329359,https://nhentai.net/api/gallery/329359,1736443,1600457187,87,"[males only, schoolboy uniform, yaoi, story ar...",[],[],[],"['chinese', 'translated']",[haruyama moto],[manga],https:

In [22]:
def preprocess_image(filename: str) -> Union[os.PathLike, None]:
    image_path = Path(filename)
    image_preproc_dir = Path(DATA_PATH) / "preprocessed" / image_path.parent
    image_preproc_path = image_preproc_dir / Path(image_path.stem + ".jpg")

    # Preprocess only those images that do not exist in
    # the destination dataset for performance reason
    if not image_preproc_path.is_file():
        try:
            image = Image.open(Path(DATA_PATH) / "raw" / image_path)
            # Normalize images to be JPEG compatible
            image = image.convert("RGB")
            image_preproc_dir.mkdir(parents=True, exist_ok=True)
            image.save(image_preproc_path, "JPEG")
        except:
            return None

    return str(PurePath(image_preproc_path))

def preprocess_images(hentai: pd.Series) -> bool:
    return [
        preprocess_image(filename)
        for filename in hentai["filenames"]
    ]

tqdm.pandas()
hentais_df = hentais_df.assign(filenames=hentais_df.progress_apply(preprocess_images, axis=1))

C:\Anaconda3\lib\site-packages\tqdm\std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 1000/1000 [00:29<00:00, 33.90it/s]


We will filter every hentais that contain at least one corrupted images, since pages have order dependencies.

In [23]:
hentais_df = hentais_df[hentais_df["filenames"].apply(all)]

In [24]:
# The metadata CSV file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "preprocessed" / "metadata.csv"
hentais_df.to_csv(
    metadata_path,
    index=False,
    header="column_names",
    columns=[
        "id",
        "title",
        "num_favorites",
        "tag",
        "group",
        "parody",
        "character",
        "language",
        "artist",
        "category",
        "num_pages",
        "filenames"
    ]
)
print("Number of preprocessed images :", len(hentais_df))

Number of preprocessed images : 999
